## Demo
Notebook to present the functionality of the EPANETUtils class made by me.

In [1]:
import json
import time
import pandas as pd
import datetime

import sys  
sys.path.insert(0, './../utils-py')
from EPANETUtils import EPANETUtils
import CompareUtils

In [2]:
file = "./../data/RaduNegru12April2021_2.2.inp"

# Setting the nodes that interest us
sensors = ["SenzorComunarzi-NatVech", "SenzorCernauti-Sebesului", "SenzorChisinau-Titulescu", "SenzorComunarzi-castanului"]
reservoirs_and_closest_junctions = [("GA-Braila", "751-B"), ("Apollo", "748-B"), 
                                    ("RaduNegruMare", "763-B"), ("RaduNegru2", "760-B")]
for reservoir, junction in reservoirs_and_closest_junctions:
    sensors.append(junction)
    

epanet_util_instance_2 = EPANETUtils(file, "PDD")

### Comparing real data with simulated 

In [3]:
simulated_data = epanet_util_instance_2.generate_pressures_at_nodes()[sensors]

hours = []
for hour in list(simulated_data.index):
    temp_str = ""
    if hour < 36000:  # 10 * 3600 after 10 hours
        temp_str += "0"
    temp_str += str(int(hour / 3600))
    hours.append(temp_str + ":00:00")
    
simulated_data.index = hours
simulated_data["SenzorComunarzi-NatVech"]

00:00:00    14.517897
01:00:00    14.558394
02:00:00    14.570760
03:00:00    14.572337
04:00:00    14.573099
05:00:00    14.575253
06:00:00    14.565536
07:00:00    14.524527
08:00:00    14.401075
09:00:00    14.282032
10:00:00    14.275187
11:00:00    14.349558
12:00:00    14.387352
13:00:00    14.444325
14:00:00    14.434622
15:00:00    14.458322
16:00:00    14.492256
17:00:00    14.475909
18:00:00    14.446707
19:00:00    14.414341
20:00:00    14.358551
21:00:00    14.367382
22:00:00    14.346522
23:00:00    14.403765
24:00:00    14.517899
Name: SenzorComunarzi-NatVech, dtype: float32

In [4]:
file_names_arr = ["2020-10-23-882360011565773_pressure.csv", "2020-10-23-882360011565772_pressure.csv",
                 "2020-10-23-882360011565771_pressure.csv", "2020-10-23-882360011565770_pressure.csv"]
dict_of_data = CompareUtils.compare_real_with_simulated_data(file_names_arr, source_dir="../data/braila_normal_sensors/")

# 8 static comparisong
dict_of_data["2020-10-23-882360011565773_pressure.csv"].to_dict()

{'C1 Pres (m)': {Timedelta('0 days 00:00:00'): 18.8,
  Timedelta('0 days 01:00:00'): 18.825,
  Timedelta('0 days 02:00:00'): 18.875,
  Timedelta('0 days 03:00:00'): 18.9,
  Timedelta('0 days 04:00:00'): 18.549999999999997,
  Timedelta('0 days 05:00:00'): 19.6,
  Timedelta('0 days 06:00:00'): 19.525,
  Timedelta('0 days 07:00:00'): 18.474999999999998,
  Timedelta('0 days 08:00:00'): 18.174999999999997,
  Timedelta('0 days 09:00:00'): 18.025,
  Timedelta('0 days 10:00:00'): 18.225,
  Timedelta('0 days 11:00:00'): 18.375,
  Timedelta('0 days 12:00:00'): 18.35,
  Timedelta('0 days 13:00:00'): 18.275,
  Timedelta('0 days 14:00:00'): 18.525,
  Timedelta('0 days 15:00:00'): 18.424999999999997,
  Timedelta('0 days 16:00:00'): 18.0,
  Timedelta('0 days 17:00:00'): 17.825,
  Timedelta('0 days 23:00:00'): 18.5}}

### Simulating leaks

In [5]:
print("Starting simulation.....")
now = time.time()

print("Generating leaks.....")
leakage_arr = [0.003]
dict_leaks2, dict_diff2 = epanet_util_instance_2.run_leakage_scenario(leaks_arr=leakage_arr, generate_diff_dict=True, 
                                                                      retrieve_specific_nodes_arr=sensors)

with open("1-leaks-dict-selected.json", 'w') as fp:
    json.dump(dict_leaks2, fp, indent=4) # 
print("To calculate it took: {}s".format(time.time() - now))

later = time.time()
print("Total duration: {}s".format(later - now))

Starting simulation.....
Generating leaks.....


EpanetException: EPANET Error 223

In [ ]:
# TODO calibrate data to pressures
print("Starting simulation.....")
now = time.time()
epanet_util_instance_1 = EPANETUtils(file, "PDD")

print("Generating leaks.....")
dict_leaks1, dict_diff1 = epanet_util_instance_1.run_leakage_scenario(leaks_arr=[0.006, 0.012], generate_diff_dict=True)

with open("1-leaks-dict-all.json", 'w') as fp:
    json.dump(dict_leaks1, fp, indent=4)
print("To calculate it took: {}s".format(time.time() - now))

later = time.time()
print("Total duration: {}s".format(later - now))

In [ ]:
"""# Files are 130MB smaller if there is no indent
with open("1-leaks-dict.json", 'w') as fp:
    json.dump(dict_leaks, fp, indent=4)

print("Leaks to file it took: {}s".format(time.time() - now))

with open("1-diff-dict.json", 'w') as fp:
    json.dump(dict_diff, fp, indent=4)""";
pd.DataFrame(dict_leaks1)